In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Download the training set
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

--2023-10-05 21:04:09--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.207, 64.233.189.207, 108.177.125.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M  29.1MB/s    in 5.7s    

2023-10-05 21:04:15 (25.0 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]



In [3]:
# Download the validation set
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

--2023-10-05 21:04:15--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.207, 64.233.189.207, 108.177.125.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘validation-horse-or-human.zip’

validation-horse-or 100%[===================>]  10.95M  9.24MB/s    in 1.2s    

2023-10-05 21:04:16 (9.24 MB/s) - ‘validation-horse-or-human.zip’ saved [11480187/11480187]



In [4]:
import zipfile

In [5]:
# Unzip training set
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')

# Unzip validation set
local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./validation-horse-or-human')

zip_ref.close()

In [6]:
import os

# Directory with training horse pictures
train_horse_dir = os.path.join('./horse-or-human/horses')

# Directory with training human pictures
train_human_dir = os.path.join('./horse-or-human/humans')

# Directory with validation horse pictures
validation_horse_dir = os.path.join('./validation-horse-or-human/horses')

# Directory with validation human pictures
validation_human_dir = os.path.join('./validation-horse-or-human/humans')

In [7]:
train_horse_names = os.listdir(train_horse_dir)


print(f'TRAIN SET HORSES: {train_horse_names[:10]}')

train_human_names = os.listdir(train_human_dir)
print(f'TRAIN SET HUMANS: {train_human_names[:10]}')

validation_horse_names = os.listdir(validation_horse_dir)
print(f'VAL SET HORSES: {validation_horse_names[:10]}')

validation_human_names = os.listdir(validation_human_dir)
print(f'VAL SET HUMANS: {validation_human_names[:10]}')

TRAIN SET HORSES: ['horse34-6.png', 'horse27-4.png', 'horse07-7.png', 'horse02-6.png', 'horse28-9.png', 'horse32-7.png', 'horse09-8.png', 'horse15-2.png', 'horse17-2.png', 'horse48-0.png']
TRAIN SET HUMANS: ['human05-13.png', 'human09-16.png', 'human07-24.png', 'human04-20.png', 'human10-00.png', 'human01-15.png', 'human08-24.png', 'human05-11.png', 'human15-28.png', 'human06-28.png']
VAL SET HORSES: ['horse4-599.png', 'horse1-384.png', 'horse5-060.png', 'horse3-011.png', 'horse5-275.png', 'horse3-498.png', 'horse1-411.png', 'horse3-217.png', 'horse2-254.png', 'horse2-201.png']
VAL SET HUMANS: ['valhuman04-11.png', 'valhuman02-15.png', 'valhuman05-14.png', 'valhuman05-08.png', 'valhuman04-05.png', 'valhuman02-10.png', 'valhuman01-13.png', 'valhuman03-07.png', 'valhuman02-09.png', 'valhuman02-20.png']


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255.)
validation_datagen = ImageDataGenerator(rescale = 1/255.)

train_generator = train_datagen.flow_from_directory(
    directory = "./horse-or-human/",
    target_size = (300, 300),
    batch_size = 128,
    class_mode = "binary"
)

validation_generator = validation_datagen.flow_from_directory(
    directory = "./validation-horse-or-human",
    target_size = (300, 300),
    batch_size = 32,
    class_mode = "binary"
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [9]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [10]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [12]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.82:
            print("\nReached 82% accuracy so cancelling training!")
            self.model.stop_training = True

In [14]:
history = model.fit(
      train_generator,

      epochs=15,
      verbose=1,
      validation_data = validation_generator,

      callbacks = myCallback())

Epoch 1/15
9/9 [==============================] - 101s 11s/step - loss: 0.9460 - accuracy: 0.5375 - val_loss: 0.6866 - val_accuracy: 0.5000
Epoch 2/15
9/9 [==============================] - 99s 11s/step - loss: 0.6412 - accuracy: 0.6008 - val_loss: 1.4750 - val_accuracy: 0.5000
Epoch 3/15
9/9 [==============================] - 100s 11s/step - loss: 0.6916 - accuracy: 0.6514 - val_loss: 0.4700 - val_accuracy: 0.8477
Epoch 4/15
9/9 [==============================] - ETA: 0s - loss: 0.3918 - accuracy: 0.8754
Reached 82% accuracy so cancelling training!
9/9 [==============================] - 101s 11s/step - loss: 0.3918 - accuracy: 0.8754 - val_loss: 0.8748 - val_accuracy: 0.7461
